In [1]:
# !/usr/bin/env python

#environment setting
import pandas as pd
import sqlite3
import re
# Library Import and Environment Setting needed for Logistic Regression

import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

plt.rc("font", size=16)
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

import os
import os.path
import glob
import re

import json
import time

from collections import defaultdict
from collections import Counter

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import nltk.data
nltk.downloader.download('vader_lexicon')
nltk.downloader.download('stopwords')

# from nltk.sentiment.vader import SentimentIntensityAnalyzer

!pip install profanity-check
from profanity_check import predict, predict_prob
# https://github.com/vzhou842/profanity-check


import argparse

!pip install coverage

C:\Users\Kyle_Shipley\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


  Using cached https://files.pythonhosted.org/packages/ff/f8/8073dd1939ee3d2ee9da977599380f30923289f477e90cacc2f0d9af286b/profanity_check-1.0.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d1/6c/6ddb21e203ff95d7080aeee2105b4f6610a02483e00d4ac950f3630969c9/scikit_learn-0.20.2-cp37-cp37m-win_amd64.whl


In [2]:
#     parser = argparse.ArgumentParser('Parses the given directory of lyrics')
#     parser.add_argument('dir_given', help='Directory of the lyrics')
#     args = parser.parse_args()
#     dir_given = args.dir_given


#Simulated 'Given' directory, needs to be changed to input from command line kss0416
dir_given = r'C:\Users\Kyle_Shipley\Documents\Columbia_Docs\IEOR 4501\ProjectTemp\TFA_Project-Alex\Lyrics'

start_time = time.time()

#Need to change path below to start in lyrics directory kss0416
os.chdir(dir_given)

raw_filenames = []
artists = []

# Get .txt files names
# Changed below line to use the lyrics directory as the current directory kss0416
for file_name in os.listdir():
    if file_name.endswith('.txt'):
        raw_filenames.append(file_name)
        artists.append(file_name.split("~")[2].split(".txt")[0])

def raw_filenames_list(str)->list:
    raw_filenames_list_ = []
    for filename in os.listdir('Lyrics'):
        if filename.endswith(str):
            raw_filenames_list_.append(filename)
    return raw_filenames_list_

def artists_list(str)->list:
    artists_list_ = []
    for filename in raw_filenames:
        if filename.endswith(str):
            artists_list_.append(filename.split("~")[2].split(".txt")[0])
    return artists_list_

def songs_list(str)->list:
    songs_list_ = []
    for filename in raw_filenames:
        if filename.endswith(str):
            songs_list_.append(filename.split("~")[1].split(".txt")[0])
    return songs_list_

def song_ordering_numbers_list(str)->list:
    song_ordering_numbers_list_ = []
    for filename in raw_filenames:
        if filename.endswith(str):
            song_ordering_numbers_list_.append(filename.split("~")[0].split(".txt")[0])
    return song_ordering_numbers_list_

def artist_s_songs_list(str)->list:
    artist_s_songs_list_ = []
    for raw_filename in os.listdir():
        if raw_filename.endswith(str+'.txt'):
            artist_s_songs_list_.append(raw_filename)
    return artist_s_songs_list_

In [4]:
# %%timeit
cell_s_t = time.time()

#Makes cleaned song dir and cleans songs to remove uneeded characters
def song_cleaning(artists):
    try:
        os.mkdir(dir_given + '/Cleaned_Songs')
    except FileExistsError:  
        pass

    for artist in set(artists):
        for song in artist_s_songs_list(artist):
            f = open(dir_given + r'/' +  song, 'rb')
            all_words = ''
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')
                all_words += this_sentence
            #remove identifiers like chorus, verse, etc
            all_words = re.sub(r'[\(\[],.*?[\)\]]', '', all_words)
            #remove empty lines
            all_words = os.linesep.join([s for s in all_words.splitlines() if s])
            f.close()
            f = open(os.path.join(dir_given + '/Cleaned_Songs', 'cleaned_' + song ), "wb")
            f.write(all_words.encode('utf-8'))
            f.close()
            
song_cleaning(artists)
cell_t = time.time()
print(cell_t - cell_s_t)

In [9]:
# %%timeit
cell_s_t = time.time()

# plt.rcParams['figure.figsize'] = (126,190)

# df = pd.DataFrame(columns=('artist', 'words'))
# i=0
            
def artist_s_cleaned_songs_list(str)->list:
    artist_s_cleaned_songs_list_ = []
    for cleaned_raw_filename in os.listdir(dir_given + '/Cleaned_Songs/'):
        if cleaned_raw_filename.endswith(str + '.txt') is True:
            if cleaned_raw_filename.endswith(str + '.txt') not in artist_s_cleaned_songs_list_:
                artist_s_cleaned_songs_list_.append(cleaned_raw_filename)
    return artist_s_cleaned_songs_list_

def word_num(artists):
    print('running')
#     i = 0
    word_num_dict = {}
    for artist in set(artists):
#         all_words = ''
#         df.loc[i] = (artist, num_words)
#         i += 1
        for song in artist_s_cleaned_songs_list(artist):
            num_words = 0
            f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')
                num_words_this = len(this_sentence.split(" "))
                num_words += num_words_this
            word_num_dict[song] = num_words
    return word_num_dict

word_num_dict = word_num(artists)
print(len(artists))

cell_t = time.time()
print(cell_t - cell_s_t)
# print(word_num_dict)

running


In [5]:
cell4s_t = time.time()

# #Adjusted below directory, and made function kss0426
# os.chdir(dir_given)

def sentiment(artists):
#     df = pd.DataFrame(columns=('artist', 'pos', 'neu', 'neg'))
    sid = SentimentIntensityAnalyzer()
    # Commas do not affect the SentimentIntensityAnalyzer() (http://www.nltk.org/howto/sentiment.html)
    i=0

    num_neutral = 0
    num_negative = 0
    num_positive = 0

    for artist in set(artists):
        for song in artist_s_cleaned_songs_list(artist):
            f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')
                comp = sid.polarity_scores(this_sentence)
                comp = comp['compound']
                if comp >= 0.5:
                    num_positive += 1
                elif comp > -0.5 and comp < 0.5:
                    num_neutral += 1
                else:
                    num_negative += 1

        num_total = num_negative + num_neutral + num_positive
        if num_total != 0:
            percent_negative = (num_negative/float(num_total))*100
            percent_neutral = (num_neutral/float(num_total))*100
            percent_positive = (num_positive/float(num_total))*100
#             df.loc[i] = (artist, percent_positive, percent_neutral, percent_negative)
#             i+=1
        else:
            percent_negative = 0
            percent_neutral = 0
            percent_positive = 0
#             df.loc[i] = (artist, percent_positive, percent_neutral, percent_negative)
#             i+=1
    
sentiment(artists)

cell4_t = time.time()
print(cell4_t - cell4s_t)

In [6]:
cell5s_t = time.time()

plt.rcParams['figure.figsize'] = (126,190)

df = pd.DataFrame(columns=('artist', 'words'))
i=0

def artist_s_cleaned_songs_list(str)->list:
    artist_s_cleaned_songs_list_ = []
    for cleaned_raw_filename in os.listdir(dir_given + '/Cleaned_Songs'):
        if cleaned_raw_filename.endswith(str + '.txt') is True:
            if cleaned_raw_filename.endswith(str + '.txt') not in artist_s_cleaned_songs_list_:
                artist_s_cleaned_songs_list_.append(cleaned_raw_filename)
    return artist_s_cleaned_songs_list_

def min_max_numfunc(artists):
    num_words_list_ = []

    for artist in set(artists):
        num_words = 0
        for song in artist_s_cleaned_songs_list(artist):
            f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')
                num_words_this = len(this_sentence.split(" "))
                num_words += num_words_this
        num_words_list_.append(num_words)

    min_num_words_list_ = min(num_words_list_)
    max_num_words_list_ = max(num_words_list_)
    return min_num_words_list_, max_num_words_list_
    
min_num_words_list_, max_num_words_list_ = min_max_numfunc(artists)

In [7]:
cell_s_t = time.time()

def regularize_num(artists):
    for artist in set(artists):
        for song in artist_s_cleaned_songs_list(artist):
            num_words = 0
            f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')
                num_words_this = len(this_sentence.split(" "))
                num_words += num_words_this
            regularization_step = (num_words - min_num_words_list_)/(max_num_words_list_ - min_num_words_list_) 
            num_words_score = 1*regularization_step + 0*(1-regularization_step)
            num_words_score = round(num_words_score,2)
    
    return num_words_score

num_words_score = regularize_num(artists)

cell_t = time.time()
print(cell_t - cell_s_t)

In [8]:
cell_s_t = time.time()

# def lexical_richness(artists):
#     plt.rcParams['figure.figsize'] = (126,190)

#     df = pd.DataFrame(columns=('artist', 'lexicalrichness'))
#     for artist in set(artists):
#         num_words = 0
#         all_words = ''
#         raw_text = ""
#         for song in artist_s_cleaned_songs_list(artist):
#             f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
#             for sentence in f.readlines():
#                 this_sentence = sentence.decode('utf-8')
#                 raw_text += this_sentence

#         words = raw_text.split(" ")
#         filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords

#         a = len(set(filtered_words))
#         b = len(words)

#lexical_richness(artists)

def unique_words(artists):
    unique_words_list_ = []
    unique_words_dict = {}  

    for artist in set(artists):
        for song in artist_s_cleaned_songs_list(artist):
            raw_text = "" 
            f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')
                raw_text += this_sentence
            words = raw_text.split(" ")
            filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords

            a = len(set(filtered_words))
            b = len(words)
            unique_words_ratio = a/float(b)
            unique_words_ratio = round(unique_words_ratio,2)
            unique_words_list_.append(unique_words_ratio)
            unique_words_dict[song] = unique_words_ratio
    return unique_words_dict

uw = unique_words(artists)

cell_t = time.time()
print(cell_t - cell_s_t)

[0.28, 0.52, 0.49, 0.26, 0.41, 0.45, 0.25, 0.23, 0.48, 0.47, 0.78, 0.79, 0.55, 0.13, 0.36, 0.72, 0.3, 0.35, 0.28, 0.33, 0.38, 0.25, 0.51, 0.18, 0.21, 0.52, 0.36, 0.49, 0.44, 0.18, 0.23, 0.59, 0.45, 0.44, 0.48, 0.41, 0.37, 0.44, 0.41, 0.48, 0.38, 0.23, 0.26, 0.29, 0.32, 0.3, 0.45, 0.22, 0.37, 0.42, 0.81, 0.47, 0.32, 0.37, 0.29, 0.33, 0.33, 0.51, 0.59, 0.77, 0.26, 0.42, 0.44, 0.45, 0.43, 0.36, 0.4, 0.45, 0.55, 0.51, 0.54, 0.28, 0.27, 0.46, 0.37, 0.44, 0.32, 0.54, 0.46, 0.48, 0.27, 0.54, 0.51, 0.26, 0.44, 0.77, 0.38, 0.42, 0.67, 0.5, 0.51, 0.53, 0.37, 0.33, 0.2, 0.53, 0.24, 0.33, 0.32, 0.49, 0.56, 0.25, 0.29, 0.31, 0.34, 0.37, 0.45, 0.46, 0.42, 0.5, 0.28, 0.25, 0.34, 0.49, 0.41, 0.35, 0.49, 0.27, 0.5, 0.63, 0.4, 0.37, 0.17, 0.56, 0.49, 0.27, 0.63, 0.43, 0.32, 0.35, 0.54, 0.45, 0.47, 0.25, 0.71, 0.5, 0.29, 0.42, 0.3, 0.25, 0.48, 0.44, 0.37, 0.39, 0.43, 0.25, 0.34, 0.3, 0.39, 0.61, 0.52, 0.21, 0.33, 0.59, 0.5, 0.39, 0.3, 0.37, 0.28, 0.45, 0.36, 0.38, 0.46, 0.16, 0.23, 0.33, 0.55, 0.55, 0.39

In [9]:
cell_s_t = time.time()


def love_ratio(artists):
    love_words_list_ = [
                       'adore', 'adores', 'adorable', 'affection', 'amour', 'angel', 'bliss', 
                       'care', 'caring', 'chocolate', 'companion', 'compassion', 'concern', 
                       'darling', 'dear', 'desire', 'devotion', 'endearment', 'family', 
                       'fondness', 'forever', 'friendship', 'fun', 'God', 'happiness', 'happy', 
                       'happily', 'heart', 'hugs', 'husband', 'infatuation', 'inspiration', 
                       'intimacy', 'joy', 'kiss', 'kisses', 
                       'love', 'loves', 'loved', 'loving', 
                       'loyalty', 'marriage', 'passion', 'relationship', 'romance', 'sex', 
                       'sweet', 'sweetheart', 'tenderness', 'trust', 'warmth', 'wife'
                        ]

    love_ratio_dict = {}
    love_ratio_list_ = []

    for artist in set(artists):
        for song in artist_s_cleaned_songs_list(artist):
            counter_for_love_words = 0
            raw_text = ""

#                 cleaned_song_name = song.split("cleaned_")[1].split("~")[1]
#                 id_ = song.split("cleaned_")[1].split("~")[0]

            f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')
                raw_text += this_sentence

            words = raw_text.split(" ")
            filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords

            for item in filtered_words:
                if item in love_words_list_:
                    counter_for_love_words += 1

            love_ratio = counter_for_love_words/len(filtered_words)
            love_ratio = round(love_ratio,4)
            love_ratio_list_.append(love_ratio)
            love_ratio_dict[song] = love_ratio

    min_love_ratio_list_ = min(love_ratio_list_)
    max_love_ratio_list_ = max(love_ratio_list_)
    
    return love_ratio_dict, min_love_ratio_list_, max_love_ratio_list_

# Creating the regularization step for the length attribute
def regularize_love_ratio(artists, love_ratio_dict, min_love_ratio_list_, max_love_ratio_list_):
    love_score_dict = {}
    for artist in set(artists):    
        for song in artist_s_cleaned_songs_list(artist):
            regularization_step = (love_ratio_dict[song] - min_love_ratio_list_)/(max_love_ratio_list_ - min_love_ratio_list_) 
            love_ratio_score = round(regularization_step,2)
            love_score_dict[song] = love_ratio_score
    return love_score_dict

love_ratio_dict, min_love_ratio_list_, max_love_ratio_list_ = love_ratio(artists)
love_score_dict = regularize_love_ratio(artists, love_ratio_dict, min_love_ratio_list_, max_love_ratio_list_)

cell_t = time.time()
print(cell_t - cell_s_t)

In [10]:
cell_s_t = time.time()

df = pd.DataFrame(columns=('artist', 'pos', 'neu', 'neg'))
sid = SentimentIntensityAnalyzer()
i=0

# os.chdir('/Users/alexpapaioannou/Dropbox/Personal Things/Studies Related/NY/COLUMBIA UNIVERSITY/IEOR/MSMSE/Courses/IEOR 4501 E 001 TOOLS FOR ANALYTICS/TOOLS_2019/Project_2019/')
os.chdir(dir_given)
num_neutral = 0
num_negative = 0
num_positive = 0
unique_words_list_ = []
# love_words_list_ = [
#                    'adore', 'adores', 'adorable', 'affection', 'amour', 'angel', 'bliss', 
# #                    'care', 'caring', 'chocolate', 'companion', 'compassion', 'concern', 
# #                    'darling', 'dear', 'desire', 'devotion', 'endearment', 'family', 
# #                    'fondness', 'forever', 'friendship', 'fun', 'God', 'happiness', 'happy', 
#                    'happily', 'heart', 'hugs', 'husband', 'infatuation', 'inspiration', 
#                    'intimacy', 'joy', 'kiss', 'kisses', 
#                    'love', 'loves', 'loved', 'loving', 
# #                    'loyalty', 'marriage', 'passion', 'relationship', 'romance', 'sex', 
# #                    'sweet', 'sweetheart', 'tenderness', 'trust', 'warmth', 'wife'
#                     ]

love_ratio_list_ = []
dict_for_json = {}
# with open('data_output.txt', 'w') as fp:
# with open('./data_output.txt', 'w') as f: 

num_neutral = 0
num_negative = 0
num_positive = 0

num_words = 0
all_words = ''
raw_text = "" 

counter_for_love_words = 0

#         if len(artist_s_cleaned_songs_list(artist)) == 1:

dict_for_json_song = {}
list_of_dicts = []

for artist in set(artists):
    for song in artist_s_cleaned_songs_list(artist):
        cleaned_song_name = song.split("cleaned_")[1].split("~")[1]
        id_ = song.split("cleaned_")[1].split("~")[0]

        dict_for_json_song["id"] = id_
        dict_for_json_song["artist"] = artist
        dict_for_json_song["title"] = cleaned_song_name

        f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
        for sentence in f.readlines():
            this_sentence = sentence.decode('utf-8')

            raw_text += this_sentence

            num_words_this = len(this_sentence.split(" "))
            num_words += num_words_this

            comp = sid.polarity_scores(this_sentence)
            comp = comp['compound']
            if comp >= 0.5:
                num_positive += 1
            elif comp > 0 and comp < 0.5:
                num_neutral += 1
            else:
                num_negative += 1

        sentence_for_profanity_check_ = [this_sentence]
        profanity_check = predict_prob(sentence_for_profanity_check_)
        profanity_score = float(' '.join(map(str, profanity_check)))
        dict_for_json_song["kids_safe"] = round(profanity_score,2)

        regularization_step = (num_words - min_num_words_list_)/(max_num_words_list_ - min_num_words_list_) 
        num_words_score = 1*regularization_step + 0*(1-regularization_step)
        num_words_score = round(num_words_score,2)

        num_total = num_negative + num_neutral + num_positive
        if num_total != 0:
            percent_negative = round((num_negative/float(num_total))*100,2)
            percent_neutral = round((num_neutral/float(num_total))*100,2)
            percent_positive = round((num_positive/float(num_total))*100,2)
#                 df.loc[i] = (artist, percent_positive, percent_neutral, percent_negative)
#                 i+=1
        else:
            percent_negative = 0
            percent_neutral = 0
            percent_positive = 0

#             words = raw_text.split(" ")
#             filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords

#             a = len(set(filtered_words))
#             b = len(words)
#             unique_words_ratio = a/float(b)
#             unique_words_ratio = round(unique_words_ratio,2)
#             unique_words_list_.append(unique_words_ratio)


#             for item in words:
#                 if item in love_words_list_:
#                     counter_for_love_words += 1
#             len_counter_for_love_words = len(love_words_list_)
#             love_ratio = counter_for_love_words/float(len_counter_for_love_words)
#             love_ratio = round(love_ratio,4)
#             love_ratio_list_.append(love_ratio) 

#             regularization_step = (love_ratio - min_love_ratio_list_)/(max_love_ratio_list_ - min_love_ratio_list_ - 16) 
#             love_ratio_score = 1*regularization_step
#             love_ratio_score = round(love_ratio_score,2)

        dict_for_json_song["love"] = love_score_dict[song]
        dict_for_json_song["mood"] = round(comp,2)
                                        #Should be word_num_score dict below
        dict_for_json_song["length"] = word_num_dict[song]
        dict_for_json_song["complexity"] = uw[song]
        #print(dict_for_json_song)
        
        list_of_dicts.append(dict_for_json_song)

#         dict_for_json.setdefault('characterizations:', list_of_dicts)
        dict_for_json.setdefault('characterizations:', []).append(dict_for_json_song)
#dict_for_json = {'characterizations:', list_of_dicts}
print(dict_for_json)

cell_t = time.time()
print(cell_t - cell_s_t)

#d = count(string.split), where string is song text
#Test for specific indevidual songs
#Assert true outputs are between 0 and one

{
      "characterizations": [

             {

                     "id" = 836,
                     "artist" = Marty Wilde,

                     "id" = 925,
                     "artist" = Rex Harrison,

                     "id" = 197,
                     "artist" = Slam Stewart,

                     "id" = 233,
                     "artist" = The McGuire Sisters,

                     "id" = 360,
                     "artist" = Fear Factory,

                     "id" = 161,
                     "artist" = Van Halen,

                     "id" = 218,
                     "artist" = Van Halen,

                     "id" = 993,
                     "artist" = Dennis Brown,

                     "id" = 853,
                     "artist" = Paul Weller,

                     "id" = 957,
                     "artist" = Billy Taylor,

                     "id" = 458,
                     "artist" = Red Lorry Yellow Lorry,

                     "id" = 111,
                     "artist" 

In [11]:
# print(dict_for_json)
print(json.dumps(dict_for_json, indent=4))

{
    "characterizations:": [
        {
            "id": "836",
            "artist": "Marty Wilde",
            "title": "Teenager In Love",
            "kids_safe": 0.11,
            "love": 0.01,
            "mood": 0.64,
            "length": 0.05,
            "complexity": 0.28
        },
        {
            "id": "925",
            "artist": "Rex Harrison",
            "title": "Why can't the English",
            "kids_safe": 0.11,
            "love": 0.0,
            "mood": 0.0,
            "length": 0.09,
            "complexity": 0.52
        },
        {
            "id": "197",
            "artist": "Slam Stewart",
            "title": "I'm in the Mood for Love",
            "kids_safe": 0.15,
            "love": 0.0,
            "mood": 0.64,
            "length": 0.02,
            "complexity": 0.49
        },
        {
            "id": "233",
            "artist": "The McGuire Sisters",
            "title": "Sugartime",
            "kids_safe": 0.12,
            "lo

In [1]:
# with open('data_output.json', 'r') as f:  
#     dict_for_json = json.load(f)
#     print(dict_for_json)

len(dict_for_json['characterizations:'])

NameError: name 'dict_for_json' is not defined

In [13]:
# def love_ratio_score(song)->float:
    
#     words = []
#     raw_text = "" 
#     counter_for_love_words = 0
    
#     love_words_list_ = [
#                    'adore', 'adores', 'adorable', 'affection', 'amour', 'angel', 'bliss', 
# #                    'care', 'caring', 'chocolate', 'companion', 'compassion', 'concern', 
# #                    'darling', 'dear', 'desire', 'devotion', 'endearment', 'family', 
# #                    'fondness', 'forever', 'friendship', 'fun', 'God', 'happiness', 'happy', 
#                    'happily', 'heart', 'hugs', 'husband', 'infatuation', 'inspiration', 
#                    'intimacy', 'joy', 'kiss', 'kisses', 
#                    'love', 'loves', 'loved', 'loving', 
# #                    'loyalty', 'marriage', 'passion', 'relationship', 'romance', 'sex', 
# #                    'sweet', 'sweetheart', 'tenderness', 'trust', 'warmth', 'wife'
#                     ]
    
#     love_ratio_list_ = []

#     f = open('Lyrics/Cleaned_Songs/' + song , 'rb')
#     for sentence in f.readlines():
#         this_sentence = sentence.decode('utf-8')
#         raw_text += this_sentence

#     words = raw_text.split(" ")
#     filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords

#     for item in words:
#         if item in love_words_list_:
#             counter_for_love_words += 1
#     len_counter_for_love_words = len(love_words_list_)
#     love_ratio = counter_for_love_words/float(len_counter_for_love_words)
#     love_ratio = round(love_ratio,4)
#     love_ratio_list_.append(love_ratio)
    
#     for artist in set(artists):   
        
#         for song in artist_s_cleaned_songs_list(artist):
#             cleaned_song_name = song.split("cleaned_")[1].split("~")[1]
#             id_ = song.split("cleaned_")[1].split("~")[0]
#             f = open('Lyrics/Cleaned_Songs/' + song , 'rb')
#             for sentence in f.readlines():
#                 this_sentence = sentence.decode('utf-8')
                
#                 raw_text += this_sentence
            
#             words = raw_text.split(" ")
#             filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords
            
#             for item in words:
#                 if item in love_words_list_:
#                     counter_for_love_words += 1
#             len_counter_for_love_words = len(love_words_list_)
#             love_ratio = counter_for_love_words/float(len_counter_for_love_words)
#             love_ratio = round(love_ratio,4)
#             love_ratio_list_.append(love_ratio) 
            
#     min_love_ratio_list_ = min(love_ratio_list_)
#     max_love_ratio_list_ = max(love_ratio_list_)

#     regularization_step = (love_ratio - min_love_ratio_list_)/(max_love_ratio_list_ - min_love_ratio_list_) 
#     love_ratio_score = 1*regularization_step + 0*(1-regularization_step)
#     love_ratio_score = round(love_ratio_score,1)
#     return love_ratio_score

In [14]:
# a = love_ratio_score('cleaned_688~I Wanna Be Loved~Buy This Song.txt')
# print(a)